In [1]:
import gym
import os
import sys
import random
import numpy as np
import pickle
import torch as T
#from dueling_dqn_torchB1 import C51DuelDQNAgentPER
from model import C51DuelDQNAgentPER
#from dueling_dqn_torchB1copy import C51DuelDQNAgentPER
#from dqn_agent1 import DQNAgent
#from utils import plot_learning_curve, make_env
from gym import wrappers

from RLStock_Utils import DataEnv, StockTradingEnvBB
import pandas as pd

from datetime import datetime
from collections import deque

import multiprocessing as mp
from multiprocessing import Pool
import gc

In [2]:
#T.cuda.memory_allocated()
T.cuda.empty_cache()
T.cuda.memory_allocated()
##############################
gc.collect()

5

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    T.manual_seed(seed)
    T.cuda.manual_seed(seed)
    T.cuda.manual_seed_all(seed)
    T.backends.cudnn.deterministic = True
    T.backends.cudnn.benchmark = True
#    T.set_float32_matmul_precision('high')  # (PyTorch 2.0+) - improves matmul ops
    
seed_everything(2021)

np.random.seed(2021)
random.seed(2021)
T.manual_seed(2021)
T.cuda.manual_seed(2021)
T.backends.cudnn.deterministic = True
T.backends.cudnn.benchmark = False

In [4]:
# number of days back in time
ndays = 25

In [5]:
# Simple ISO 8601 timestamped logger
def log(s):
  print('[' + str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')) + '] ' + s)

In [6]:
data_path = os.path.join(sys.prefix, 'data')
dataenv = DataEnv(save_tuple_buffer=False, path=data_path, fname="last_10_years", ndays=ndays)
#dataenv = DataEnv(save_tuple_buffer=False, path=data_path, fname="last_10_years", ndays=ndays)
#dataenv = DataEnv(save_tuple_buffer=False)

In [7]:
#dataenv.process()
#dataenv.process_save(fname="t0one_sma_last_10_years")
dataenv.process_load(fname="sma_last_10_years")
#dataenv.process_load(fname="t0one_sma_last_10_years")

loading C:\Users\bjorn\miniconda3\envs\rlstock\data\tuplebuffer_train_sma_last_10_years_25d.pickle
loading C:\Users\bjorn\miniconda3\envs\rlstock\data\tuplebuffer_test_sma_last_10_years_25d.pickle
loading C:\Users\bjorn\miniconda3\envs\rlstock\data\date_ranges_sma_last_10_years_25d.pickle


In [8]:
#env.date_ranges.popitem() # removes last entry to dictionary
dataenv.ticks = dataenv.date_ranges.keys()
###################################################################################################################
exclude_list = ['OXY', 'CZR', 'OKE', 'KDP', 'NCLH', 'TRGP', 'APA', 'SBL', 'COV', 'CPWR', 'CVG'] # ticks with negative BLL and KLL values
# Filter out the excluded tickers from self.ticks
dataenv.ticks = [tick for tick in dataenv.ticks if tick not in exclude_list]
###################################################################################################################
len(list(dataenv.ticks)), dataenv.ticks

(573,
 ['1COV.DE',
  'AAL',
  'AAPL',
  'AAP',
  'ABBV',
  'ABT',
  'ACGL',
  'ACN',
  'ADBE',
  'ADI',
  'ADM',
  'ADP',
  'ADS.DE',
  'ADSK',
  'ADT',
  'AEE',
  'AEP',
  'AES',
  'AFL',
  'AIG',
  'AIR.PA',
  'AIZ',
  'AJG',
  'AKAM',
  'ALB',
  'ALGN',
  'ALK',
  'ALLE',
  'ALL',
  'ALV.DE',
  'AMAT',
  'AMCR',
  'AMD',
  'AME',
  'AMGN',
  'AMG',
  'AMP',
  'AMT',
  'AMZN',
  'ANET',
  'ANSS',
  'AON',
  'AOS',
  'APD',
  'APH',
  'APTV',
  'ARE',
  'ASML',
  'ATO',
  'AVB',
  'AVGO',
  'AVY',
  'AWK',
  'AXON',
  'AXP',
  'AYI',
  'AZN',
  'AZO',
  'A',
  'BAC',
  'BALL',
  'BAS.DE',
  'BAX',
  'BAYN.DE',
  'BA',
  'BBY',
  'BDX',
  'BEI.DE',
  'BEN',
  'BG',
  'BHF',
  'BIIB',
  'BIO',
  'BKNG',
  'BKR',
  'BK',
  'BLDR',
  'BLK',
  'BMS',
  'BMW.DE',
  'BMY',
  'BNR.DE',
  'BRO',
  'BR',
  'BSX',
  'BWA',
  'BXP',
  'BX',
  'CAG',
  'CAH',
  'CARR',
  'CAT',
  'CBK.DE',
  'CBOE',
  'CBRE',
  'CB',
  'CCEP',
  'CCI',
  'CCL',
  'CDNS',
  'CDW',
  'CE',
  'CFG',
  'CF',
  'CHD',


In [9]:
env = StockTradingEnvBB(dataenv, 0, ndays=ndays)
#Z25env = StockTradingEnvBB(dataenv.tbuf_train, dataenv.tbuf_test, dataenv.ticks)
#env = gym.make('CartPole-v1')

In [10]:
best_score = -np.inf
load_checkpoint = False
n_games = 5000

# Number of environments to run sequentially
n_envs = 64

In [11]:
agent = C51DuelDQNAgentPER(gamma=0.99, epsilon=0.002*n_envs, lr=4*6.25e-5,
#agent = C51DuelDQNAgentPER(gamma=0.994, epsilon=0.002*n_envs, lr=4*6.25e-5,
#agent = C51DuelDQNAgentPER(gamma=0.997, epsilon=0.002*n_envs, lr=4*6.25e-5,
#agent = C51DuelDQNAgentPER(gamma=0.99, epsilon=0.002*n_envs, lr=1*6.25e-5,
#805 agent = C51DuelDQNAgentPER(gamma=0.99, epsilon=0.002*n_envs, lr=4*6.25e-5,
#agent = C51DuelDQNAgentPER(gamma=0.99, epsilon=0.002*n_envs, lr=1e-3,
                input_dims=((1,73,5)),
                b_step=0,
                n_step=4,#8,#5,#3,#3,
                n_actions=7, mem_size=2 ** 20, eps_min=0.0,
                batch_size=256, replace=1000, nenvs=n_envs, eps_dec=5e-5,
                ndays = ndays,
#                batch_size=64, replace=1000, nenvs=n_envs, eps_dec=5e-5,
                chkpt_dir='models/')

if load_checkpoint:
    print('loading models')
    agent.load_models()

...allocating a PrioritizedReplayBuffer


In [12]:
print(agent.q_next)

EfficientC51DuelingDeepQNetworkCV2J23(
  (groupz_pos): CustomNorm1d()
  (group1): GlobalGroupNorm()
  (group2): CustomNorm2d()
  (group3): GlobalGroupNorm()
  (group4): CustomNorm2d()
  (group_state): Conv2d(9, 9, kernel_size=(1, 1), stride=(1, 1), groups=9)
  (conv_mainA): Sequential(
    (0): Conv2d(19, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    (1): CustomNorm2d()
    (2): ReLU()
    (3): ResNetBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
      (bn1): CustomNorm2d()
      (relu): ReLU()
      (conv2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
      (bn2): CustomNorm2d()
    )
    (4): SE2DIdentity(
      (fc1): Linear(in_features=32, out_features=5, bias=True)
      (fc2): Linear(in_features=5, out_features=32, bias=True)
    )
    (5): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=32)
    (6): Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    (7): Custom

In [13]:
!nvidia-smi

Mon Jan 19 11:17:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.83                 Driver Version: 581.83         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P3             12W /   70W |    4081MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
class Elo:
    def __init__(self,k,g=1,homefield = 100):
        self.ratingDict = {}
        self.k = k
        self.g = g
        self.homefield = homefield

    def addPlayer(self,name,rating = 1500):
        self.ratingDict[name] = rating

    def gameOver(self, winner, loser, winnerHome):
        if winnerHome:
            result = self.expectResult(self.ratingDict[winner] + self.homefield, self.ratingDict[loser])
        else:
            result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser]+self.homefield)

        self.ratingDict[winner] = self.ratingDict[winner] + (self.k*self.g)*(1 - result)  
        self.ratingDict[loser] 	= self.ratingDict[loser] + (self.k*self.g)*(0 - (1 -result))

    def expectResult(self, p1, p2):
        exp = (p2-p1)/400.0
        return 1/((10.0**(exp))+1)

In [15]:
eloMarket, eloHuman, eloAI = [], [], []
eloscore = Elo(k=20)
eloscore.addPlayer("Market", rating = 100)
eloscore.addPlayer("Human", rating = 100)
eloscore.addPlayer("AI", rating = 100)

In [16]:
best_score_eval = -1000

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Financial Times color scheme
ft_colors = {
    'ft_pink': '#FFF1E5',
    'ft_black': '#000000',
    'ft_blue': '#0D559F',
    'ft_lightblue': '#85C1E9',
    'ft_green': '#28A745',
    'ft_yellow': '#FFC000',
    'ft_red': '#DC3545'
}

def test_new(tick, step, nstep=200, bal=10000, metrics=True):
    price, real_price, adi, pdi, ndi, rsi = [], [], [], [], [], []
    hist, bma, bul, bll = [], [], [], []
    
    agent.q_eval.eval()
    agent.q_next.eval()

    tbuf = env.tbuf_test
    env.btraining = False
    env.set_step(step=step, balance=bal, shares_held=[0, 0], shares_held_value=[0, 0], tick=tick)

    # initial observation (first action is implicitly HOLD)
    observation = env.pred_step(training=False)  # self.current_step = step
    hist.append(' ')  # index 0

    # ---- init context for new regime stats path ----
    d_med  = getattr(agent.q_eval, "d_med_raw", None)
    d_slow = getattr(agent.q_eval, "d_slow_raw", None)
#    print(f'd_med={d_med}, d_slow={d_slow}')

#999    d_med = getattr(agent, "d_med_raw", None)
#999    d_slow = getattr(agent, "d_slow_raw", None)

    med_ctx = np.zeros(d_med, dtype=np.float32) if d_med is not None else None
    slow_ctx = np.zeros(d_slow, dtype=np.float32) if d_slow is not None else None

    for i in range(nstep):
        agent.q_eval.eval()
        agent.q_next.eval()

        # ---- pick action, optionally using regime context ----
        if med_ctx is not None or slow_ctx is not None:
            # agent.pred_action should accept med_ctx / slow_ctx in new code
#            print(f'med_ctx={med_ctx},slow_ctx={slow_ctx}')
            action = agent.pred_action(
                observation,
                med_ctx=med_ctx,
                slow_ctx=slow_ctx,
            )
        else:
            # backward compatible if pred_action ignores context
            action = agent.pred_action(observation)
 

        # ---- step env ----
        observation, reward, done, info = env.step(action)

        # update context for *next* decision if available
        if "medium_stats" in info:
            arr = info["medium_stats"]
            if d_med is not None:
                med_ctx = np.asarray(arr, dtype=np.float32).reshape(-1)
                if med_ctx.shape[0] > d_med:
                    med_ctx = med_ctx[:d_med]
                elif med_ctx.shape[0] < d_med:
                    tmp = np.zeros(d_med, dtype=np.float32)
                    tmp[:med_ctx.shape[0]] = med_ctx
                    med_ctx = tmp
        if "slow_stats" in info:
            arr = info["slow_stats"]
            if d_slow is not None:
                slow_ctx = np.asarray(arr, dtype=np.float32).reshape(-1)
                if slow_ctx.shape[0] > d_slow:
                    slow_ctx = slow_ctx[:d_slow]
                elif slow_ctx.shape[0] < d_slow:
                    tmp = np.zeros(d_slow, dtype=np.float32)
                    tmp[:slow_ctx.shape[0]] = slow_ctx
                    slow_ctx = tmp

        # fetch TA / price data
        [(curcls, nxtcls, curbma, curbul, curbll, zbma, zbul, zbll,
          zkma, zkul, zkll, zcls, zrsi, zadi, zpdi, zndi,
          zhgh, zlow, zopn, zem12, zem26)] = tbuf.get_indices((tick, step + i))
        
        real_price.append(curcls)
        adi.append(zadi[-1] * 100)
        pdi.append(zpdi[-1] * 100)
        ndi.append(zndi[-1] * 100)
        rsi.append(zrsi[-1] * 100)

        bma.append(curbma)
        bul.append(curbul)
        bll.append(curbll)
        
        # build action history (skip last index because we don't have a plotted price there)
        if i < nstep - 1:
            if action == 0:
                hist.append(' ')
            elif action == 1:
                hist.append('a')   # buy small
            elif action == 2:
                hist.append('b')   # buy medium
            elif action == 3:
                hist.append('c')   # buy large
            elif action == 4:
                hist.append('s')   # sell small
            elif action == 5:
                hist.append('t')   # sell medium
            elif action == 6:
                hist.append('u')   # sell large
            else:
                hist.append('?')

        if done:
            # you *can* break early if you want, but then adjust plotting length accordingly
            pass

    # ---- scoring vs market ----
    xcprice = env.cprice[-1] / env.cprice[1]
    if np.sum(env.total_shares_bought1) > 0:
        xnet_worth = (env.balance + env.shares_held[0] * env.cprice[-1]) / 10000
    else:
        xnet_worth = env.net_worth / 10000

    if xcprice > xnet_worth:
        if xcprice > 1:
            eloscore.gameOver('Human', 'AI', 0)
        else:
            eloscore.gameOver('Market', 'AI', 0)
    else:
        if xnet_worth > 1:
            eloscore.gameOver('AI', 'Human', 0)
        else:
            eloscore.gameOver('Market', 'Human', 0)

    if metrics:
        eloMarket.append(eloscore.ratingDict['Market'])
        eloHuman.append(eloscore.ratingDict['Human'])
        eloAI.append(eloscore.ratingDict['AI'])
        title = f'{tick} price vs model  {xcprice:,.2f} / {xnet_worth:,.2f} {step:03d} {n_steps:08d} eloAI={eloAI[-1]:.1f}'
        print(f'{tick} Human:{eloHuman[-1]:.1f} AI:{eloAI[-1]:.1f}')
    else:
        title = f'{tick} price vs model  {xcprice:,.2f} / {xnet_worth:,.2f} {step:03d} {n_steps:08d}'
        
    pricex = np.array(real_price[:])
    adix = np.array(adi[:])
    pdix = np.array(pdi[:])
    ndix = np.array(ndi[:])
    rsix = np.array(rsi[:])
    
    bmax = np.array(bma[:])
    bulx = np.array(bul[:])
    bllx = np.array(bll[:])
    
    fig = plt.figure(figsize=(14, 4), dpi=100)
    plt.rcParams['figure.figsize'] = [14, 4]
    plt.title(title, loc='center', color='b')
    plt.grid(color='#C5C9C7')
    plt.plot(pricex, color='darkblue', linewidth=0.7)
    plt.plot(bmax, alpha=0.7, linewidth=0.5)
    plt.plot(bulx, alpha=0.7, linewidth=0.5)
    plt.plot(bllx, alpha=0.7, linewidth=0.5)
    plt.fill_between(np.linspace(0, nstep-1, nstep), bulx, bllx, color='grey', alpha=0.1)

    # ---- draw action markers ----
    for i in range(len(hist)):
        ch = hist[i]
        if ch in ('a', 'b', 'c', 'd'):
            color = 'b'   # buys -> blue
        elif ch in ('s', 't', 'u', 'v'):
            color = 'm'   # sells -> magenta
        elif ch == ' ':
            color = 'g'   # hold -> green
        else:
            color = 'k'   # unknown / debug
        if i < len(pricex):  # guard in case of early done
            plt.text(i, pricex[i], ch, ha='center', color=color)

    # secondary axis for RSI / ADI
    ax2 = plt.twinx()
    ax2.plot(rsix, color='#C0C0C0', alpha=0.7, linewidth=0.5)
    ax2.plot(adix, color='#C0C0C0', alpha=0.7, linewidth=0.5)
    
    path = './img/'
    fig.savefig(f'{path}img_{n_steps:08d}_{tick}.png')
    plt.close(fig)
    
    env.btraining = True

    return xnet_worth / xcprice


In [18]:
%matplotlib inline
import matplotlib.pyplot as plt

# Financial Times color scheme
ft_colors = {
    'ft_pink': '#FFF1E5',
    'ft_black': '#000000',
    'ft_blue': '#0D559F',
    'ft_lightblue': '#85C1E9',
    'ft_green': '#28A745',
    'ft_yellow': '#FFC000',
    'ft_red': '#DC3545'
}

def test(tick, step, nstep=200, bal=10000, metrics=True):
    price, real_price, adi, pdi, ndi, rsi, hist, close_macd, close_signal, bma, bul, bll, ema12, ema26 = [], [], [], [], [], [], [], [], [], [], [], [], [], []
    
    agent.q_eval.eval()
    agent.q_next.eval()

    tbuf = env.tbuf_test
    env.btraining = False
    env.set_step(step=step, balance=bal, shares_held=[0, 0], shares_held_value=[0, 0], tick=tick)

#    action, volume = agent.pred_action(observation, training=False)
    observation = env.pred_step(training=False)  # self.current_step = step

    # first action is implicitly "hold"
    hist.append(' ')  # index 0

    for i in range(nstep):
        agent.q_eval.eval()
        agent.q_next.eval()

        # given current observation, predict action in next step
#        action, volume = agent.pred_action(observation, training=False)
        action = agent.pred_action(observation, training=False)

        # take action with reward, step+, next observation
#        observation, reward, done, info = env.step((action, volume))
        observation, reward, done, info = env.step(action)

        # fetch TA / price data
        [(curcls, nxtcls, curbma, curbul, curbll, zbma, zbul, zbll,
          zkma, zkul, zkll, zcls, zrsi, zadi, zpdi, zndi,
          zhgh, zlow, zopn, zem12, zem26)] = tbuf.get_indices((tick, step + i))
        
        real_price.append(curcls)
        adi.append(zadi[-1] * 100)
        pdi.append(zpdi[-1] * 100)
        ndi.append(zndi[-1] * 100)
        rsi.append(zrsi[-1] * 100)

        bma.append(curbma)
        bul.append(curbul)
        bll.append(curbll)
        
        # build action history (skip last index because we don't have a plotted price there)
        if i < nstep - 1:
            # 7-action mapping:
            # 0 = hold
            # 1 = buy small
            # 2 = buy medium
            # 3 = buy large
            # 4 = sell small
            # 5 = sell medium
            # 6 = sell large
            if action == 0:
                hist.append(' ')
            elif action == 1:
                hist.append('b')   # buy small
            elif action == 2:
                hist.append('c')   # buy medium
            elif action == 3:
                hist.append('d')   # buy large
            elif action == 4:
                hist.append('s')   # sell small
            elif action == 5:
                hist.append('t')   # sell medium
            elif action == 6:
                hist.append('u')   # sell large
            else:
                # fallback – shouldn't happen
                hist.append('?')

    # ---- scoring vs market ----
    xcprice = env.cprice[-1] / env.cprice[1]
    if np.sum(env.total_shares_bought1) > 0:
        xnet_worth = (env.balance + env.shares_held[0] * env.cprice[-1]) / 10000
    else:
        xnet_worth = env.net_worth / 10000

    if xcprice > xnet_worth:
        if xcprice > 1:
            eloscore.gameOver('Human', 'AI', 0)
        else:
            eloscore.gameOver('Market', 'AI', 0)
    else:
        if xnet_worth > 1:
            eloscore.gameOver('AI', 'Human', 0)
        else:
            eloscore.gameOver('Market', 'Human', 0)

    if metrics:
        eloMarket.append(eloscore.ratingDict['Market'])
        eloHuman.append(eloscore.ratingDict['Human'])
        eloAI.append(eloscore.ratingDict['AI'])
        title = f'{tick} price vs model  {xcprice:,.2f} / {xnet_worth:,.2f} {step:03d} {n_steps:08d} eloAI={eloAI[-1]:.1f}'
        print(f'{tick} Human:{eloHuman[-1]:.1f} AI:{eloAI[-1]:.1f}')
    else:
        title = f'{tick} price vs model  {xcprice:,.2f} / {xnet_worth:,.2f} {step:03d} {n_steps:08d}'
        
    pricex = np.array(real_price[:])
    adix = np.array(adi[:])
    pdix = np.array(pdi[:])
    ndix = np.array(ndi[:])
    rsix = np.array(rsi[:])
    
    bmax = np.array(bma[:])
    bulx = np.array(bul[:])
    bllx = np.array(bll[:])
    
    fig = plt.figure(figsize=(14, 4), dpi=100)
    plt.rcParams['figure.figsize'] = [14, 4]
    plt.title(title, loc='center', color='b')
    plt.grid(color='#C5C9C7')
    plt.plot(pricex, color='darkblue', linewidth=0.7)
    plt.plot(bmax, alpha=0.7, linewidth=0.5)
    plt.plot(bulx, alpha=0.7, linewidth=0.5)
    plt.plot(bllx, alpha=0.7, linewidth=0.5)
    plt.fill_between(np.linspace(0, nstep-1, nstep), bulx, bllx, color='grey', alpha=0.1)

    # ---- draw action markers ----
    for i in range(len(hist)):
        ch = hist[i]
        if ch in ('a', 'b', 'c', 'd'):
            color = 'b'   # buys -> blue
        elif ch in ('s', 't', 'u', 'v'):
            color = 'm'   # sells -> magenta
        elif ch == ' ':
            color = 'g'   # hold -> green (or you can skip drawing)
        else:
            color = 'k'   # unknown / debug
        plt.text(i, pricex[i], ch, ha='center', color=color)

    # secondary axis for RSI / ADI
    ax2 = plt.twinx()
    ax2.plot(rsix, color='#C0C0C0', alpha=0.7, linewidth=0.5)
    ax2.plot(adix, color='#C0C0C0', alpha=0.7, linewidth=0.5)
    
    path = './img/'
    fig.savefig(f'{path}img_{n_steps:08d}_{tick}.png')
    plt.close(fig)
    
    env.btraining = True

    return xnet_worth / xcprice


In [19]:
#nw = test('T', 123, nstep=200, metrics=False) 

In [20]:
fname = 'test' + '_' + 'agent_env_name' + '_lr' + '0.0001' + '_' + str(n_games) + 'games'
figure_file = 'plots/' + fname + '.png'

In [21]:
n_steps = 0
ascores, scores, s_scores, eps_history, steps_array = deque(maxlen=10000), deque(maxlen=10), deque(maxlen=10000), deque(maxlen=10000), deque(maxlen=10000)

In [22]:
n_games = 10 * 30 * 3300#3 * 28000#60000#15000#1100

In [23]:
since_last, rewardc = [], []
best_since_last = 0.

In [24]:
max_steps = 80001#80001#65#1001#65#10001#80001#25001#80001#64+32#64#80001#64#20001 # Max number of steps before learning starts; Per Rainbow, 80k with PER works
bootstrap = False#False # Set to True when saving initial randomized buffer after max_steps; Only used once 

In [25]:
if bootstrap:
    agent.memory = pickle.load(open('./pickle/buf_160.pkl', 'rb'))  # load from "buf.pkl"
    agent.memory_n = pickle.load(open('./pickle/buf_n_160.pkl', 'rb'))  # load from "buf_n.pkl"
    n_steps = max_steps
    agent.epsilon = 1 - (n_steps / 250000)

In [26]:
interrupted = False

In [27]:
####################################### Vecntor based Environments

In [28]:
from copy import deepcopy
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
#from concurrent.futures import ThreadPoolExecutor, as_completed
from stable_baselines3.common.vec_env import DummyVecEnv

In [29]:
class DummyVecEnvNoFlatten_new(DummyVecEnv):
    """
    Slightly modified version of stable_baselines3's DummyVecEnv.
    - Observations are not flattened.
    - Supports actions as either:
        * int          -> env.step(action)
        * (action, v)  -> env.step(action, volume_fraction=v)
    """

    def step_async(self, actions):
        """Queue the action for later execution."""
        self.actions = actions

    def step_wait(self):
        obs_list = [None] * self.num_envs
        for env_idx in range(self.num_envs):

            act = self.actions[env_idx]

            # support (action, volume_fraction) pairs
            if isinstance(act, (tuple, list, np.ndarray)) and len(act) == 2:
                a, vol = act
                obs_, self.buf_rews[env_idx], self.buf_dones[env_idx], truncated, self.buf_infos[env_idx] = \
                    self.envs[env_idx].step(int(a), volume_fraction=float(vol))
            else:
                # old behavior: plain discrete action
                obs_, self.buf_rews[env_idx], self.buf_dones[env_idx], truncated, self.buf_infos[env_idx] = \
                    self.envs[env_idx].step(act)

            obs_list[env_idx] = obs_

        return obs_list, self.buf_rews.copy(), self.buf_dones.copy(), self.buf_infos

    def reset(self):
        obs_list = [None] * self.num_envs
        for env_idx in range(self.num_envs):
            obs_list[env_idx] = self.envs[env_idx].reset()[0]
        return obs_list

    def reset_at(self, env_idx):
        obs, _ = self.envs[env_idx].reset()
        return obs

    def train(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = True

    def test(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = False


In [30]:
class DummyVecEnvNoFlatten(DummyVecEnv):
    """
    Slightly modified version of stable_baselines3's DummyVecEnv. The main difference is that observations are not
    flattened before they are returned. This is done to make it work with our lazy frame-stacking class further below.
    """

    def step_async(self, actions):
        """Queue the action for later execution."""
        self.actions = actions
        
    def step_wait(self):
        obs_list = [None] * self.num_envs
        for env_idx in range(self.num_envs):
            
            obs_, self.buf_rews[env_idx], self.buf_dones[env_idx], truncated, self.buf_infos[env_idx] = \
                self.envs[env_idx].step(self.actions[env_idx])
            obs_list[env_idx] = obs_
            
        return obs_list, self.buf_rews.copy(), self.buf_dones.copy(), self.buf_infos

    def reset(self):
        obs_list = [None] * self.num_envs
        for env_idx in range(self.num_envs):
            obs_list[env_idx] = self.envs[env_idx].reset()[0]
        return obs_list

    def reset_at(self, env_idx):
        obs, _ = self.envs[env_idx].reset()
        return obs

    def train(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = True

    def test(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = False


In [31]:
class DummyVecEnvNoFlatten_orig(DummyVecEnv):
    """
    Slightly modified version of stable_baselines3's DummyVecEnv. The main difference is that observations are not
    flattened before they are returned. This is done to make it work with our lazy frame-stacking class further below.
    """

    def step_async(self, actions):
        """Queue the action for later execution."""
        self.actions = actions
        
    def step_wait(self):
        obs_list = []
        for env_idx in range(self.num_envs):
            
            obs_, self.buf_rews[env_idx], self.buf_dones[env_idx], truncated, self.buf_infos[env_idx] = \
                self.envs[env_idx].step(self.actions[env_idx])
            
            obs_list.append(obs_)
            
        return obs_list, np.copy(self.buf_rews), np.copy(self.buf_dones), deepcopy(self.buf_infos)

    def reset(self):
        obs_list = []
        for env_idx in range(self.num_envs):
            obs, _ = self.envs[env_idx].reset()
            obs_list.append(obs)
        return obs_list

    def reset_at(self, env_idx):
        obs, _ = self.envs[env_idx].reset()
        return obs

    def train(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = True

    def test(self):
        for env_idx in range(self.num_envs):
            self.envs[env_idx].btraining = False


In [32]:
# Function to create environments
def make_env(dataenv, rank):
    def _init():
        env = StockTradingEnvBB(dataenv, rank, ndays=ndays)  # Replace this with your environment
        return env
    return _init

# Number of environments to run sequentially
#n_envs = 64

# Creating the vectorized environments using DummyVecEnv
envs = DummyVecEnvNoFlatten([make_env(dataenv, i) for i in range(n_envs)])

C:\Users\bjorn\miniconda3\envs\rlstock\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [33]:
scores = deque(maxlen=1000)
best_score = -10.

In [34]:
#max_steps = 1000

In [ ]:
#### Reset the environment
envs.train()
observations = envs.reset()
agent.q_next.eval()
print(observations[0])

# Initialize per-environment episode reward trackers
episode_rewards = [0.0] * n_envs  
scores = []  # To store completed episode scores
obs_list = [None] * n_envs

while n_steps <= 225000000:
    agent.q_eval.eval()

    obs_array = np.stack(observations)
    actions = agent.choose_actions(obs_array, n_envs=n_envs, show_action=False, training=True).tolist()
    
    # for DummyVecEnvNoFlatten, you can pass tuple arrays
    envs.step_async(actions)

    if n_steps >= max_steps:
        agent.q_eval.train()
        for train_cnt in range(2):
            if train_cnt > 0:
                agent.q_eval.reset_noise()
            agent.learn(n_steps)

    observations_, rewards, dones, infos = envs.step_wait()
    
    for j, (observation, action, reward, observation_, done, info) in enumerate(
            zip(observations, actions, rewards, observations_, dones, infos)):
        
        # Store transition as before
        agent.store_transition_news(observation, action, reward, observation_, done, info, j)
        
        # Accumulate reward for each environment
        episode_rewards[j] += reward
        
        # If an episode is done, record its score and reset the reward accumulator
        if done:
            scores.append(episode_rewards[j])
            episode_rewards[j] = 0.0
            observation_ = envs.reset_at(j)
        
        # Update the observation list for the next step
        obs_list[j] = observation_
    
    observations = obs_list

    # Calculate the current score as the mean of the last n_envs completed episodes
    if len(scores) >= n_envs:
        current_score = np.array(scores)[-n_envs:].mean()
    else:
        current_score = np.array(scores).mean() if scores else 0.0

    if current_score >= best_score and n_steps > 1000:
        best_score = current_score

    if n_steps % 16000 == 0:
        if len(agent.loss_c) > 1:
            log('## n_step %0d score %.3f best score %.3f loss %.5f' % 
                (n_steps, current_score, best_score, agent.loss_c[-1]))
        else:
            log('## n_step %0d score %.3f best score %.3f' % 
                (n_steps, current_score, best_score))
                
    if n_steps > 1000 and n_steps % 1000000 == 0:
        log('## evaluating')
        nscore_eval = []
        for tick in range(10) :
            tick = np.random.choice(list(env.ticks))
            xrange = env.date_ranges[tick][1] - 202
            if xrange > 1:
                step = random.randint(ndays+1, xrange)
                nw = test(tick, step, nstep=200)
                nscore_eval.append(nw)
                
        nw = test('T', 153, nstep=200, metrics=False)

        score_eval = np.mean(nscore_eval)
        if score_eval > best_score_eval:
            best_score_eval = score_eval

        print('score_eval=%.3f best_score_eval=%.3f'% (score_eval,best_score_eval))
        env.render()
        envs.train()
        
    n_steps += n_envs


[[[ 1.0000000e+00]
  [ 4.9822646e-01]
  [ 4.9856105e-01]
  [ 4.9949145e-01]
  [ 5.0102049e-01]
  [ 5.0226259e-01]
  [ 5.0318342e-01]
  [ 5.0403947e-01]
  [ 5.0475055e-01]
  [ 5.0603795e-01]
  [ 5.0706512e-01]
  [ 5.0818801e-01]
  [ 5.0920516e-01]
  [ 5.0930190e-01]
  [ 5.0919896e-01]
  [ 5.0849068e-01]
  [ 5.0679439e-01]
  [ 5.0364339e-01]
  [ 4.9875745e-01]
  [ 4.9439627e-01]
  [ 4.9002844e-01]
  [ 4.8617253e-01]
  [ 4.8076516e-01]
  [ 4.7527105e-01]
  [ 4.6901387e-01]
  [ 4.6210036e-01]]

 [[ 1.0000000e+00]
  [ 5.0860673e-01]
  [ 5.1559120e-01]
  [ 5.1838493e-01]
  [ 5.2068317e-01]
  [ 5.1613194e-01]
  [ 5.0856155e-01]
  [ 5.0865173e-01]
  [ 5.0932759e-01]
  [ 5.0108141e-01]
  [ 5.0594807e-01]
  [ 5.0630850e-01]
  [ 5.0851649e-01]
  [ 5.0243318e-01]
  [ 4.9855793e-01]
  [ 4.9869311e-01]
  [ 4.9184391e-01]
  [ 4.7426996e-01]
  [ 4.5228010e-01]
  [ 4.5507383e-01]
  [ 4.4741341e-01]
  [ 4.5466831e-01]
  [ 4.3362471e-01]
  [ 4.2416176e-01]
  [ 4.1834891e-01]
  [ 4.0951693e-01]]

 [[ 1.00

In [36]:
data = env.tbuf_train.get('T', 1000)

In [36]:
#data

In [38]:
(curcls, nxtcls, curbma, curbul, curbll, 
 zbma, zbul, zbll, zkma, zkul, zkll, 
 zcls, zrsi, zadi, zpdi, zndi, 
 zhgh, zlow, zopn, zem12, zem26) = data[0]


In [40]:
env.TRADE_PENALTY = 0.012
env.PROFIT_CONVEXITY_K = 0.07

In [41]:
for e in envs.envs:
    e.TRADE_PENALTY = 0.012
    e.PROFIT_CONVEXITY_K = 0.07
    print(e.TRADE_PENALTY)

0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012


In [ ]:
print(f"modz.. {agent.groupr.running_mean} {agent.groupr.running_var}")

In [36]:
np.log(2 + 1e-3)

0.6936470556015963

In [ ]:
agent.q_eval.linext[0].weight.device, agent.q_eval.group1.running_mean_gr.device

In [ ]:
rewards
# mod1.. 0.5021314024925232 0.04668758809566498
# mod2.. 0.2344733476638794 0.08063570410013199
# mod3.. 0.24490664899349213 0.09989308565855026
# mod4.. 0.5229998826980591 0.12699458003044128
# mem .. 0.05433722957968712 0.10341408103704453

In [ ]:
agent.lr
#print(self.obs_buf.device, self.obs_buf.index_select(0, idx_t).device)
#agent.memory.mean_rews, agent.memory.var_rews

In [43]:
nw = test('GOOG', 153, nstep=200, bal=10000, metrics=False) 
nw = test('AAPL', 153, nstep=200, bal=10000, metrics=False) 
nw = test('T', 153, nstep=200, bal=10000, metrics=False)

In [39]:
nw = test('T', 153, nstep=200)

T Human:80.0 AI:-133.1


In [44]:
for tick in range(10) :
    tick = np.random.choice(list(env.ticks))
    xrange = env.date_ranges[tick][1] - 202
    if xrange > 1:
        step = random.randint(ndays+1, xrange)
        print(tick, step, ndays, xrange)
#        nw = test(tick, step, nstep=200)


MMM 346 25 537
TSCO 188 25 537
KHC 79 25 497
COF 61 25 537
GEN 408 25 537
MBG.DE 92 25 545
FOX 148 25 219
TDY 40 25 537
REG 478 25 537
VLO 252 25 537


In [ ]:
##################################################################################################### Concurrency

In [26]:
# --- setup & state ---
n_steps = 0
ascores, scores, s_scores, eps_history, steps_array = deque(maxlen=10000), deque(maxlen=10), deque(maxlen=10000), deque(maxlen=10000), deque(maxlen=10000)

n_games = 10 * 30 * 3300
since_last, rewardc = [], []
best_since_last = 0.

max_steps = 80001
bootstrap = False

if bootstrap:
    agent.memory   = pickle.load(open('./pickle/buf_160.pkl', 'rb'))
    agent.memory_n = pickle.load(open('./pickle/buf_n_160.pkl', 'rb'))
    n_steps = max_steps
    agent.epsilon = 1 - (n_steps / 250000)

interrupted = False

In [ ]:
from copy import deepcopy

# >>> CHANGED: use official import path for SubprocVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv

# (keep your DummyVecEnv import if you still use it elsewhere)
# from stable_baselines3.common.vec_env import DummyVecEnv


# -------------------------------
# VecEnv factory (picklable thunk)
# -------------------------------
def make_env(dataenv, rank):
    def _init():
        # NOTE: pass ndays from your notebook scope
        env = StockTradingEnvBB(dataenv, rank, ndays=ndays)
        return env
    return _init


# -------------------------------
# Create SubprocVecEnv
# -------------------------------
# n_envs = <set this earlier in your notebook>
envs = SubprocVecEnv([make_env(dataenv, i) for i in range(n_envs)])

# >>> CHANGED: toggle training flag inside each subprocess
envs.set_attr("btraining", True)

# Reset → ndarray of shape (n_envs, C, T, 1)
observations = envs.reset()
agent.q_next.eval()
print(observations[0].shape)  # sanity check (C, T, 1)

# Per-env trackers
episode_rewards = [0.0] * n_envs
scores = []
obs_batch_next = None

best_score = -10.

# (Optional) a single local env for ad-hoc evaluation blocks later
# Ensure `env` exists if you reference it below in your eval code.
try:
    env
except NameError:
    env = StockTradingEnvBB(dataenv, sd=999, ndays=ndays)

# -------------------------------
# Main training loop
# -------------------------------
while n_steps <= 225_000_000:
    agent.q_eval.eval()

    # >>> CHANGED: observations already batched; no np.stack needed
    actions = agent.choose_actions(
        observations, n_envs=n_envs, show_action=False, training=True
    )
    actions = np.asarray(actions, dtype=np.int64)  # (n_envs,)

    # Non-blocking step then wait
    envs.step_async(actions)
    observations_, rewards, dones, infos = envs.step_wait()

    # Store transitions & accumulate rewards
    for j in range(n_envs):
        agent.store_transition_news(observations[j], int(actions[j]), float(rewards[j]),
                                    observations_[j], bool(dones[j]), infos[j], j)
        episode_rewards[j] += float(rewards[j])

    # Handle finished episodes: move scores & reset those envs
    # Try SB3 2.x fast-path
    reset_done_supported = False
    try:
        new_obs = envs.reset_done(dones)   # SB3 >= 2.0
        reset_done_supported = True
        if new_obs is not None:
            observations_[dones] = new_obs[dones]
    except AttributeError:
        pass

    if not reset_done_supported:
        # SB3 1.x fallback: reset individual sub-envs
        done_idxs = np.where(dones)[0]
        if len(done_idxs):
            # record episode scores
            for idx in done_idxs:
                scores.append(episode_rewards[idx])
                episode_rewards[idx] = 0.0
            # reset and place new observations
            reset_obs_list = envs.env_method("reset", indices=done_idxs)
            for idx, ob in zip(done_idxs, reset_obs_list):
                observations_[idx] = ob

    # Advance obs
    observations = observations_

    # Score tracking (same logic)
    if len(scores) >= n_envs:
        current_score = np.array(scores)[-n_envs:].mean()
    else:
        current_score = (np.array(scores).mean() if scores else 0.0)

    if current_score >= best_score and n_steps > 1000:
        best_score = current_score

    if n_steps % 16000 == 0:
        if len(agent.loss_c) > 1:
            log('## n_step %0d score %.3f best score %.3f loss %.5f' %
                (n_steps, current_score, best_score, agent.loss_c[-1]))
        else:
            log('## n_step %0d score %.3f best score %.3f' %
                (n_steps, current_score, best_score))

    # -------- Periodic evaluation (unchanged) --------
    if n_steps > 1000 and n_steps % 1_000_000 == 0:
        log('## evaluating')
        nscore_eval = []
        # ensure `env` exists; using single-process env for eval
        for tick in range(10):
            tick = np.random.choice(list(env.ticks))
            xrange = env.date_ranges[tick][1] - 202
            if xrange > 1:
                step = random.randint(ndays+1, xrange)
                nw = test(tick, step, nstep=200)
                nscore_eval.append(nw)

        nw = test('T', 153, nstep=200, metrics=False)

        score_eval = np.mean(nscore_eval) if nscore_eval else 0.0
        if score_eval > best_score:
            best_score = score_eval

        print('score_eval=%.3f best_score_eval=%.3f' % (score_eval, best_score))
        try:
            env.render()
        except Exception:
            pass
        envs.set_attr("btraining", True)  # resume training flag in workers

    # Step counter (still adds n_envs per vectorized step)
    n_steps += n_envs